In [1]:
import sys
import os
from utils import *

/home/ljaffeux/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-06-04 09:30:02.094614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749022202.106171  613178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749022202.109716  613178 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 09:30:02.123251: I tensorflow/core/platform/cpu_f

In [2]:
import re
campaign = 'MAESTRO'
INS = 'IMU2'  # Can be None
data_dir = '../../Data/Raw'  # Path to the base data directory

if not campaign=='MAESTRO':
    INS = ''
# List all folders inside `data_dir` that contain the campaign name
matching_folders = [f for f in os.listdir(data_dir)
                    if os.path.isdir(os.path.join(data_dir, f)) and campaign in f]

# Filter further by INS if it's set and multiple matches exist
if INS and len(matching_folders) > 1:
    matching_folders = [f for f in matching_folders if INS in f]

# Get the final folder (if any)
if matching_folders:
    selected_folder = os.path.join(data_dir, matching_folders[0])
    print(f"Selected folder: {selected_folder}")
else:
    print("No matching folder found.")

preseg_dir = '../../Data/Pre_Segmentation'
# List all folders inside `preseg_dir` that contain the campaign name
matching_folders = [f for f in os.listdir(preseg_dir)
                    if os.path.isdir(os.path.join(preseg_dir, f)) and campaign in f]
if matching_folders:
    selected_segmentation = os.path.join(preseg_dir, matching_folders[0])
    print(f"Selected segmentation: {selected_segmentation}")
    presegmentation = True
else:
    print("No pre segmentation found. It will be done during the processing.")
    presegmentation = False
    # Do pre-segmentation instead. 
if not presegmentation:
    print('Assuming simple conditions ascents, descents, and turns')
    # Minimum segment length in seconds

leng_min = 270
mean_w= []
mean_alt = []
for f in os.listdir(selected_folder):
    print(f)
    full_path = os.path.join(selected_folder, f)
    if not f.endswith('.nc'):
        print(f'Skipping {f}')
        continue
    tab_segments = []
    # Extract flight number
    match = re.search(r'as(\d+)', f)
    if match:
        number = int(match.group(1))
        
    df_25=load_raw_data(full_path)

    if not presegmentation:

        # filter take off and landing
        df_25 = df_25[df_25['ALTI_INS']>(df_25['ALTI_INS'].min()+10)]
        # filter ascents and descents
        df_25['ALTITUDE'] = df_25['ALTI_INS'].rolling(window=25, center=True).mean()
        df_25 = df_25[df_25['ALTITUDE'].diff(periods=5*25).abs() < 10]
        # filter turns
        df_25 = df_25[df_25['ROLL'].abs() <2]
        df_25 = df_25[df_25['HEADING'].diff(periods=5*25).abs() <2]
        # Segment based on time gaps (assuming data is at ~25 Hz)
        dt = df_25.index.to_series().diff().dt.total_seconds().fillna(0)
        gap_idx = dt > 5  # If gap between samples is > 5 sec, we assume a break

        segment_ids = gap_idx.cumsum()
        grouped = df_25.groupby(segment_ids)

        for i, group in grouped:
            duration = (group.index[-1] - group.index[0]).total_seconds()
            if duration >= leng_min:
                tab_segments.append(group)
    else:
        # Find the matching yaml file 
        matching_yaml = [g for g in os.listdir(selected_segmentation) if str(number) in g]
        yaml_data = read_yaml(os.path.join(selected_segmentation,matching_yaml[0]))
        for seg in yaml_data['segments']:
            if seg['kind'] == 'leg':
                if (seg['end'] - seg['start']).total_seconds() >= leng_min:
                    tab_segments.append(df_25.loc[seg['start']:seg['end']].copy())

    subsegments = []
    for n,subsegment in enumerate(tab_segments):
        # Compute average wind values
        name_seg = 'default'
        score = 100
        subsegments.append({
                            'name': name_seg + '_' + str(n + 1),
                            'start': subsegment.index[0],
                            'end': subsegment.index[-1],
                            'score': float(score),
                        })
    os.makedirs('../Data/Segmentation_default/'+campaign +'/'+ INS, exist_ok = True)
    write_yaml_subsegments(number, subsegments, '../Data/Segmentation_default/'+campaign +'/'+ INS)

FileNotFoundError: [Errno 2] No such file or directory: '../../Data/Raw'